<a href="https://colab.research.google.com/github/viduliyanage7/ML-CW/blob/main/cw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("your_dataset.csv")
df.head()
df.info()
df.describe()